# 使用TensorFlow实现卷积神经网络LENET

用于MNist数据库识别的卷积神经网络Lenet的定义如下：
![CNN](http://personal.ie.cuhk.edu.hk/~ccloy/project_target_code/images/fig3.png)


In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf
import numpy as np

mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
learning_rate = 0.001
batch_size = 128
num_step = 500
display_step = 10

num_input = 784
num_classes = 10
dropout = 0.75

X = tf.placeholder(dtype=tf.float32, shape=[None, num_input])
Y = tf.placeholder(dtype=tf.float32, shape=[None, num_classes])
keep_prob = tf.placeholder(dtype=tf.float32)

In [3]:
Weights = {
    'wc1':tf.Variable(tf.random_normal([5,5,1,32])),
    'wc2':tf.Variable(tf.random_normal([5,5,32, 64])),
    'wd1':tf.Variable(tf.random_normal([7 * 7 * 64, 1024])),
    'out':tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'bc1':tf.Variable(tf.random_normal([32])),
    'bc2':tf.Variable(tf.random_normal([64])),
    'bd1':tf.Variable(tf.random_normal([1024])),
    'out':tf.Variable(tf.random_normal([num_classes]))
}

In [4]:
# 定义一些函数

def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],padding='SAME')

def conv_net(x, weights, biases, dropout):
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [5]:
# 定义模型
logits = conv_net(x=X, weights= Weights, biases= biases, dropout=keep_prob)
predict = tf.nn.softmax(logits=logits)

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels= Y))

optimizer = tf.train.AdamOptimizer(learning_rate)

train_op = optimizer.minimize(loss_op)

# 评估模型
correct_pred = tf.equal(tf.argmax(input=predict, axis=1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [6]:
with tf.Session() as sess:
    sess.run(init)
    
    for step in range(1, num_step+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        
        sess.run(train_op, feed_dict={X : batch_x, Y: batch_y, keep_prob: dropout})
        
        if step % display_step == 0 or step == 1:
            loss, acc = sess.run([loss_op, accuracy], feed_dict= {X:batch_x,
                                                                  Y: batch_y,
                                                                  keep_prob: 1.0})
            print("Step: {}, Minibatch Loss = {:.4f}, Training Accuracy{:.3f}".format(step, loss, acc))
            
    print("Optimizer Finished!")
    print("Testing Accuracy {}".format(sess.run(accuracy,
                                                feed_dict={X:mnist.test.images[:256],
                                                                     Y:mnist.test.labels[:256], 
                                                                     keep_prob:1.0})
                                      )
         )

Step: 1, Minibatch Loss = 59519.9180, Training Accuracy0.078
Step: 10, Minibatch Loss = 22991.8379, Training Accuracy0.344
Step: 20, Minibatch Loss = 9658.2949, Training Accuracy0.445
Step: 30, Minibatch Loss = 4544.7725, Training Accuracy0.734
Step: 40, Minibatch Loss = 5345.0732, Training Accuracy0.742
Step: 50, Minibatch Loss = 4269.1738, Training Accuracy0.742
Step: 60, Minibatch Loss = 3415.9460, Training Accuracy0.797
Step: 70, Minibatch Loss = 1954.6401, Training Accuracy0.867
Step: 80, Minibatch Loss = 1826.6162, Training Accuracy0.852
Step: 90, Minibatch Loss = 2280.2886, Training Accuracy0.859
Step: 100, Minibatch Loss = 1404.0552, Training Accuracy0.898
Step: 110, Minibatch Loss = 1771.6758, Training Accuracy0.883
Step: 120, Minibatch Loss = 2225.1218, Training Accuracy0.883
Step: 130, Minibatch Loss = 2580.0942, Training Accuracy0.883
Step: 140, Minibatch Loss = 800.0142, Training Accuracy0.922
Step: 150, Minibatch Loss = 1600.0521, Training Accuracy0.906
Step: 160, Minibat